In [1]:

import os
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split



In [2]:

seq_length=2
x_data_dim=4
batch_size=100
min_max_normalization_flag=True

data_dir = '../dataset'
fname = os.path.join(data_dir, 'data-02-stock_daily.csv')
df = pd.read_csv(fname)
dataset=df.copy()
ori_Y=dataset.pop("Close")
ori_X=dataset.copy()




In [3]:

X_train, X_test, Y_train, Y_test = train_test_split(ori_X,ori_Y, test_size=0.2, shuffle=False)
X_train, X_val, Y_train, Y_val= train_test_split(X_train,Y_train, test_size=0.2, shuffle=False)


In [4]:

## 데이터의 min , max, mean, std 값 구하기.
dataset_stats = X_train.describe()
dataset_stats = dataset_stats.transpose()

## data normalization
def min_max_norm(x):
  return (x - dataset_stats['min']) / (dataset_stats['max'] - dataset_stats['min'])

def standard_norm(x):
  return (x - dataset_stats['mean']) / dataset_stats['std']

if min_max_normalization_flag==True:
    min_max_norm_train_data = min_max_norm(X_train)
    min_max_norm_val_data = min_max_norm(X_val)
    min_max_norm_test_data = min_max_norm(X_test)

    data_gen_train=tf.keras.preprocessing.sequence.TimeseriesGenerator(min_max_norm_train_data.values.tolist(), Y_train.values.tolist(),
                                                                        length=seq_length, sampling_rate=1,
                                                                        batch_size=batch_size)
    data_gen_val=tf.keras.preprocessing.sequence.TimeseriesGenerator(min_max_norm_val_data.values.tolist(), Y_val.values.tolist(),
                                                                       length=seq_length, sampling_rate=1,
                                                                       batch_size=batch_size)
    data_gen_test=tf.keras.preprocessing.sequence.TimeseriesGenerator(min_max_norm_test_data.values.tolist(), Y_test.values.tolist(),
                                                                       length=seq_length, sampling_rate=1,
                                                                       batch_size=batch_size)
else:
    data_gen_train = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_train.values.tolist(),Y_train.values.tolist(),
                                                                   length=seq_length, sampling_rate=1,
                                                                   batch_size=batch_size)
    data_gen_val = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_val.values.tolist(),Y_val.values.tolist(),
                                                                   length=seq_length, sampling_rate=1,
                                                                   batch_size=batch_size)
    data_gen_test = tf.keras.preprocessing.sequence.TimeseriesGenerator(X_test.values.tolist(),Y_test.values.tolist(),
                                                                        length=seq_length, sampling_rate=1,
                                                                        batch_size=batch_size)


In [5]:

input_Layer = tf.keras.layers.Input(shape=(seq_length, x_data_dim))
x=tf.keras.layers.LSTM(20,activation='tanh')(input_Layer) ##LSTM
x=tf.keras.layers.Dense(20,activation='relu')(x)
x=tf.keras.layers.Dense(10,activation='relu')(x)
Out_Layer=tf.keras.layers.Dense(1,activation=None)(x)
model = tf.keras.Model(inputs=[input_Layer], outputs=[Out_Layer])
model.summary()



Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2, 4)]            0         
_________________________________________________________________
lstm (LSTM)                  (None, 20)                2000      
_________________________________________________________________
dense (Dense)                (None, 20)                420       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 2,641
Trainable params: 2,641
Non-trainable params: 0
_________________________________________________________________


In [6]:

loss_function=tf.keras.losses.mean_squared_error
optimize=tf.keras.optimizers.Adam(learning_rate=0.001)
metric=tf.keras.metrics.mean_absolute_error
model.compile(loss=loss_function,
              optimizer=optimize,
              metrics=[metric])

history = model.fit(
      data_gen_train,
      validation_data=data_gen_val,
      steps_per_epoch=len(X_train)/batch_size,
      epochs=1000,
      validation_freq=1,
)
print(model.evaluate(data_gen_test))


Epoch 1/1000


UnknownError:    Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[functional_1/lstm/PartitionedCall]] [Op:__inference_train_function_3134]

Function call stack:
train_function -> train_function -> train_function


In [ ]:

test_data_X, test_data_Y=data_gen_test[0]
prediction_Y=model.predict(test_data_X).flatten()
Y_test=test_data_Y.flatten()

visual_y=[]
visual_pre_y=[]
for i in range(len(prediction_Y)):
    label = Y_test[i]
    prediction = prediction_Y[i]
    print("실제가격: {:.3f}, 예상가격: {:.3f}".format(label, prediction))
    visual_y.append(label)
    visual_pre_y.append(prediction)

time = range(1, len(visual_y) + 1)
plt.plot(time, visual_y, 'r', label='ture')
plt.plot(time, visual_pre_y, 'b', label='prediction')
plt.title('stock prediction')
plt.xlabel('time')
plt.ylabel('value')
plt.legend()
plt.show()

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()